In [ ]:
import torch
import string
from datasets import load_from_disk, load_dataset
from transformers import BertForMaskedLM, BertTokenizer
from transformers import AdamW
from tqdm import tqdm
from multigpu import main
import torch.multiprocessing as mp

In [ ]:
MODEL = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(MODEL)
model = BertForMaskedLM.from_pretrained(MODEL)
MAX_LEN = 128
CLS = 101
SEP = 102
MASKED = 103
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 2
LEARNING_RATE = 1e-5
EPOCHS = 3
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

valid_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }
device = torch.device('cuda')

In [ ]:
chosen_masking_strategy = 'end_tokens'

# Helpers

### Masking helpers

In [ ]:
def assure_CLS_and_SEP_false(id_tensor, true_false_tensor):
    """
    Receives a tensor of ids and a tensor of masking, with
    True and False values. Returns a copy of the true_false_tensor
    where every CLS and SEP position is for sure set to False.

    Doesn't mutate any input value and it's free of aliasing.
    """
    ans = true_false_tensor.detach().clone()
    ans = ans & (id_tensor != CLS) & (id_tensor != SEP)
    return ans

In [ ]:
def mask_random_positions(tensor, k = 0.15):
    """
    Receives a tensor and returns
    a new tensor of the same shape randomly filled
    with True's at the rate k.
    """
    shape = tensor.shape
    rand = torch.rand(shape)
    return (rand < k)

In [ ]:
def mask_end_positions(tensor_ids):
  """
  Receives a tensor and returns
  a tensor of the same shape filled with False's
  and a single True, at the end.
  """
  shape = tensor_ids.shape
  masks = torch.zeros(shape) != 0

  for i in range(0, shape[0]):
    for j in range(0, masks[i].shape[0]):
      if tensor_ids[i][j] == SEP:
        masks[i][j-1] = True
  return masks

In [ ]:
def mask_nothing(tensor):
    """
    Receives a tensor and returns a tensor
    of the same shape filled with False's.
    """
    shape = tensor.shape
    return torch.full(shape, False)

In [ ]:
def mask(masking_strategy, tensor):
    shape = tensor.shape
    mask_arr = masking_strategy(tensor)
    mask_arr = assure_CLS_and_SEP_false(tensor, mask_arr)
    rows = shape[0]

    def indices_to_mask(idx):
        """
        Returns a list containing all positions in the
        idx-th row of mask_arr that have to be masked.

        0 <= idx < rows has to be satisfied.
        """
        to_mask_positions = mask_arr[idx].nonzero()
        return torch.flatten(to_mask_positions).tolist()

    for i in range(rows):
        selection = indices_to_mask(i)
        tensor[i, selection] = MASKED

In [ ]:
def save_labels_and_ids(sentences, path, masking_strategy):
    sentences = [sentence for sentence in sentences if sentence]
    
    inputs = tokenizer(sentences, return_tensors = 'pt', max_length = MAX_LEN, \
                        padding = True, truncation = True)
        
    inputs['labels'] = inputs['input_ids'].detach().clone()

    mask(masking_strategy, inputs['input_ids'])

    torch.save({
        'labels': inputs['labels'],
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask']
    }, f'{path}')

In [ ]:
available_masking_strategies = {}
available_masking_strategies['end_token'] = mask_end_positions 
available_masking_strategies['random_tokens'] = mask_random_positions

### Utils

In [ ]:
def send_to_device(arg):
    return arg.to(device)

# Putting everything together

In [ ]:
dt = load_dataset('carolmou/random-sentences')

In [ ]:
dt

In [ ]:
train = set(dt['train']['correct_text'])
test = set(dt['test']['correct_text'])

In [ ]:
print(len(train), len(test))

In [ ]:
train = list(train)
test = list(test)

In [ ]:
masking_function = available_masking_strategies[chosen_masking_strategy]

for type, sentences in [('train', train), ('test', test)]:
    print(f'{type}:')
    print(f'# sentences: {len(sentences)}')
    print(f'small sample: {sentences[:3]}\n')

    save_labels_and_ids(sentences, f'../../data/{chosen_masking_strategy}_{type}_tokens.pt', \
                        masking_function)

In [ ]:
def create_dataloader(path, params):

    class MyDataset(torch.utils.data.Dataset):
        def __init__(self, input):
            self.input = input

        def __getitem__(self, idx):
            return { key: torch.tensor(val[idx]) for key, val in self.input.items() }

        def __len__(self):
            return len(self.input['input_ids'])

    """
    # uncomment the following lines if you don't want
    # to sanity check
    amount_to_print = 5
    print(f"printing first {amount_to_print}:") 
    for id, label in zip(inputs['input_ids'][:amount_to_print], inputs['labels'][:amount_to_print]):
        print(f"id: {''.join(tokenizer.decode(id))}")
        print(f"label: {''.join(tokenizer.decode(label))}")
    """

    inputs = torch.load(path) 
    return torch.utils.data.DataLoader( MyDataset(inputs), **params)

In [ ]:
masking_strategy = available_masking_strategies[chosen_masking_strategy]
train_dataloader = create_dataloader('../../data/random_tokens_train_tokens', train_params)

### Training

We'll implement early stopping, that will print the best model if the next "early_stopping_patience" models are worse than it.

In [ ]:
best_loss = 100 # best loss that the current best model yielded
epochs = 3

In [ ]:
model = send_to_device(model)

In [ ]:
opt = torch.optim.AdamW(model.parameters(), lr = LEARNING_RATE)

In [ ]:
def run_epoch(num_epoch=0):
    """ 
    Returns the loss yielded by the model's current state.
    Modifies the state of the model, leaving the state
    that yields the returned loss.

    Receives a num_epoch that will be printed in the progress
    feedback.
    """
    loop = tqdm(train_dataloader, leave=True)
    average_loss = 0
    
    for batch in loop:
        opt.zero_grad()

        input_ids = batch['input_ids']
        labels =batch['labels']
        attention_mask = batch['attention_mask']

        input_ids = send_to_device(input_ids)
        labels = send_to_device(labels)
        attention_mask = send_to_device(attention_mask)

        output = model(input_ids, labels=labels, attention_mask = attention_mask)
        
        loss = output.loss
        average_loss += loss.item()
        loss.backward()
        opt.step()
       
        loop.set_description(f'Epoch {num_epoch}')
        loop.set_postfix(loss = loss.item())

    qtd = len(train_dataloader)

    return average_loss/qtd

In [ ]:
for i in range(epochs):
    loss = run_epoch(i)
    print(f'Current loss is {loss}')

In [ ]:
torch.save(best_model, path+'best_model')

### Accuracy

In [ ]:
from next_sentence_prediction.masked_llm.next_word_algorithms.priority_queue.predictions import get_all_predictions
model.to('cpu')
get_all_predictions('Hoje eu vou comer', tokenizer, model, 2, verbose=True)

In [ ]:
def remove_ponctuation(sentence):
  ponctuation_array = ['.', '!', '?', ';', '...']

  chars = [char for char in sentence]

  while chars and chars[-1] in ponctuation_array:
    chars = chars[:-1]

  sentence = ''.join(chars)

  return sentence

In [ ]:
raw_sentences_no_points = [ remove_ponctuation(sentence) for sentence in raw_sentences ]

In [ ]:
def remove_last_word(sentence):
  """
  Receives a sentence and return a tuple where the first
  element is the sentence without the last word and the
  second element is the last word itself.
  """
  words = sentence.split()
  return " ".join(words[:-1]), words[-1]

In [ ]:
def add_masked_token(sentence):
    return sentence + " <mask>"

In [ ]:
sentences = [sentence for sentence in raw_sentences_no_points if sentence.split()]

In [ ]:
desired_acc = 15

In [ ]:
# for visuals
loop = tqdm(sentences, leave=True)

processed_up_to_now = 0

accuracies = [0] * desired_acc

for sentence in loop:

    last_word_removed, last_word = remove_last_word(sentence)
    last_word_removed = add_masked_token(last_word_removed)

    try:
        bert = get_all_predictions(last_word_removed, desired_acc)
    except:
        # if i fell here, the sentence reached the maximum amount
        # of tokens, so we'll skip it
        continue
    if last_word in bert:
        ix = bert.index(last_word)
        accuracies[ix] += 1

    processed_up_to_now += 1


In [ ]:
file = open(path+'accuracy.txt', 'w')

In [ ]:
up_to_now = 0
for top, acc_val in enumerate(accuracies):
    up_to_now += acc_val 
    percentage = (up_to_now/processed_up_to_now)*100
    str = f"{top+1}: {percentage}%"
    file.write(str+'\n')
    print(str)
file.close()